In [1]:
import pandas as pd
import numpy as np
import random
import os
from autogluon.tabular import TabularPredictor
import warnings

# 경고 무시
warnings.filterwarnings('ignore')

# ==========================================
# 0. 시드 고정 (재현성 확보)
# ==========================================
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42)

# ==========================================
# 1. 데이터 로드
# ==========================================
# 경로가 맞는지 확인해주세요.
train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')
target = '임신 성공 여부'

# ==========================================
# 2. 데이터 전처리 (누수 방지: 행 단위 처리만 수행)
# ==========================================

def preprocess_data(df):
    df = df.copy()
    
    # 1. 횟수 관련 컬럼 매핑 (문자열 -> 수치형)
    count_cols = [
        '총 시술 횟수', '클리닉 내 총 시술 횟수', 
        '총 임신 횟수', '총 출산 횟수',
        'IVF 시술 횟수', 'DI 시술 횟수',
        'IVF 임신 횟수', 'DI 임신 횟수',
        'IVF 출산 횟수', 'DI 출산 횟수'
    ]
    
    def map_count_str(x):
        if pd.isna(x): return np.nan
        x = str(x)
        if '6회 이상' in x: return 6
        try:
            return int(x.replace('회', ''))
        except:
            return np.nan

    for col in count_cols:
        df[col] = df[col].apply(map_count_str)
        
    # 2. 나이 전처리: 범주형 그대로 두되, 순서형 정보(Ordinal)로 변환
    # AutoGluon은 문자열도 잘 처리하지만, 나이는 순서가 중요하므로 매핑합니다.
    age_map = {
        '만18-34세': 0, '만35-37세': 1, '만38-39세': 2, 
        '만40-42세': 3, '만43-44세': 4, '만45-50세': 5, 
        '알 수 없음': -1 # 결측/알수없음은 별도 수치로
    }
    # 매핑되지 않는 값은 -1로 처리 (안전장치)
    df['나이_코드'] = df['시술 당시 나이'].map(age_map).fillna(-1)
    
    # 3. 배아 생성 주요 이유 단순화
    def clean_reason(x):
        if pd.isna(x): return 'Unknown'
        x = str(x)
        if '시술용' in x: return 'Treatment'
        if '기증' in x: return 'Donation'
        if '저장' in x: return 'Storage'
        return 'Other'
    
    df['배아_생성_이유_단순'] = df['배아 생성 주요 이유'].apply(clean_reason)
    
    return df

# ==========================================
# 3. 파생변수 생성 (Feature Engineering)
# ==========================================

def create_features(df):
    df = df.copy()
    
    # --- A. 결측치 처리 (AutoGluon은 결측치를 잘 다루므로 0이나 임의값 채우기 최소화) ---
    # 수치형 변수들의 NaN은 0으로 채우는 게 논리적으로 타당한 경우만 채웁니다 (예: 시술 횟수)
    fill_zero_cols = [
        '총 생성 배아 수', '이식된 배아 수', '미세주입된 난자 수', 
        '수집된 신선 난자 수', '해동 난자 수'
    ]
    # 존재하는 컬럼만 처리
    cols_to_fill = [c for c in fill_zero_cols if c in df.columns]
    df[cols_to_fill] = df[cols_to_fill].fillna(0)

    # --- B. 효율성 및 비율 지표 (Zero Division 방지) ---
    epsilon = 1e-6
    
    # 난자 관련 합계
    oocyte_cols = ['수집된 신선 난자 수', '혼합된 난자 수', '해동 난자 수']
    valid_oocyte_cols = [c for c in oocyte_cols if c in df.columns]
    df['총_난자_수'] = df[valid_oocyte_cols].sum(axis=1)
    
    # 1. 배아 생성 효율 (난자 대비 배아)
    df['배아_생성_효율'] = df['총 생성 배아 수'] / (df['총_난자_수'] + epsilon)
    
    # 2. 이식 효율 (생성된 배아 중 얼마나 이식했나)
    df['배아_이식_비율'] = df['이식된 배아 수'] / (df['총 생성 배아 수'] + epsilon)
    
    # 3. 미세주입 성공률
    if '미세주입된 난자 수' in df.columns and '미세주입에서 생성된 배아 수' in df.columns:
        df['미세주입_성공률'] = df['미세주입에서 생성된 배아 수'] / (df['미세주입된 난자 수'] + epsilon)

    # --- C. 과거 성공 이력 (중요) ---
    df['시술_대비_임신_성공률'] = df['총 임신 횟수'] / (df['총 시술 횟수'] + epsilon)
    df['시술_대비_출산_성공률'] = df['총 출산 횟수'] / (df['총 시술 횟수'] + epsilon)
    
    # --- D. 상호작용 (도메인 지식) ---
    # 나이가 많을수록 이식 배아 수가 중요함
    df['나이_x_이식배아'] = df['나이_코드'] * df['이식된 배아 수']
    
    # --- E. 불임 원인 합계 ---
    infertility_cols = [col for col in df.columns if '불임 원인' in col]
    df['불임_원인_총합'] = df[infertility_cols].sum(axis=1) # 1과 0으로 되어있다고 가정
    
    # --- F. 중요 컬럼 삭제 방지 및 정리 ---
    drop_cols = ['ID', '시술 시기 코드', '난자 해동 경과일', '배아 해동 경과일',
                 'PGS 시술 여부', 'PGD 시술 여부', '착상 전 유전 검사 사용 여부', '임신 시도 또는 마지막 임신 경과 연수'] 
    df = df.drop([c for c in drop_cols if c in df.columns], axis=1)
    
    return df

# 전처리 및 파생변수 적용
train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)

train_df = create_features(train_df)
test_df = create_features(test_df)

# ==========================================
# 4. 범주형 변수 타입 명시 (pd.get_dummies 대체)
# ==========================================
# AutoGluon이 범주형 변수를 인식하도록 object 타입 컬럼을 category로 변환
# Test set의 통계를 쓰지 않으므로 누수 아님
cat_cols = train_df.select_dtypes(include=['object']).columns
for col in cat_cols:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

# ==========================================
# 5. 모델 학습 (AutoGluon)
# ==========================================

# GPU 사용 여부 체크 (없으면 CPU 전체 사용)
ag_args_fit = {
    'num_gpus': 1 if pd.api.types.is_list_like(os.environ.get('CUDA_VISIBLE_DEVICES')) else 0
}

predictor = TabularPredictor(
    label=target,
    eval_metric='roc_auc', 
    path='ag_models_improved', # 경로 변경
    problem_type='binary'
).fit(
    train_data=train_df,
    presets='best_quality', # 가장 성능이 좋은 설정
    time_limit=7200,        # 2시간 (충분함)
    num_stack_levels=3,     # 스태킹 레벨 상향 (성능 향상 핵심)
    num_bag_folds=10,       # 앙상블 폴드 수 상향 (안정성 확보)
    ag_args_fit=ag_args_fit,
    # excluded_model_types=['KNN'] # 속도가 너무 느리면 KNN 제외 고려
)

# ==========================================
# 6. 추론 및 저장
# ==========================================

# predict_proba로 확률 예측
pred_probs = predictor.predict_proba(test_df)
# 1번 클래스(성공)의 확률만 추출
final_probs = pred_probs.iloc[:, 1]

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:40 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6000
CPU Count:          10
Pytorch Version:    2.9.1
CUDA Version:       CUDA is not available
GPU Count:          WARNING: Exception was raised when calculating GPU count (AssertionError)
Memory Avail:       7.03 GB / 16.00 GB (44.0%)
Disk Space Avail:   185.19 GB / 460.43 GB (40.2%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=10, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether

In [2]:
# --- 1. 리더보드 (오름차순 정렬) ---
# 학습 데이터 내에서의 Validation Score 확인
lb = predictor.leaderboard(train_df, silent=True)
display(lb.head())

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr_BAG_L1,0.923365,0.727789,roc_auc,1.505685,6.731623,11.985702,1.505685,6.731623,11.985702,1,True,4
1,RandomForestGini_BAG_L1,0.922081,0.726805,roc_auc,1.511207,6.785319,675.066578,1.511207,6.785319,675.066578,1,True,3
2,ExtraTreesGini_BAG_L1,0.902709,0.729825,roc_auc,1.436919,6.585830,9.850467,1.436919,6.585830,9.850467,1,True,6
3,WeightedEnsemble_L2,0.760003,0.739601,roc_auc,10.245007,13.084049,4265.621937,0.006016,0.044137,708.779908,2,True,7
4,LightGBM_BAG_L1,0.749083,0.738982,roc_auc,3.205471,2.406518,8.858772,3.205471,2.406518,8.858772,1,True,2


In [ ]:
# --- 2. 피처 중요도 ---
fi = predictor.feature_importance(data=train_df.sample(n=min(5000, len(train_df)), random_state=42))
display(fi)

In [ ]:
# # --- 3. 제출 파일 생성 ---
# import datetime
# submission = pd.read_csv('../Data/sample_submission.csv')
# submission['probability'] = final_probs.values

# # 현재 시간 가져오기 (예: 0206_1031)
# now = datetime.now().strftime('%m%d_%H%M')
# file_name = f"{now}_submission.csv"
# submission.to_csv(file_name, index=False)

# print(f"학습 및 예측이 완료되었습니다. 결과가 {file_name}에 저장되었습니다.")